# ROI readers

> This module contains all code responsible for reading region of interests (ROI) files:

In [ ]:
#| default_exp readers/rois

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

from typing import Dict, List, Any
from pathlib import Path
import numpy as np
from shapely.geometry import Polygon
import roifile

from findmycells.core import DataReader
from findmycells.database import Database

In [ ]:
#| export

class ROIReaders(DataReader):
    """ 
    Return the roi(s) as shapely.geometry.Polygon(s) in a nested dictionary with structure: {plane_id: {roi_id: Polygon}}
    In case plane-specific ROIs are required / requested at some point, 
    having the additional level that enables the reference to plane_id(s) should foster the implementation.
    The current implementation, however, only supports the use of ROIs for all planes - the corresponding plane_id is hence: 'all_planes'
    Ultimately, this file_id specific dictionary can then be integrated into the 'rois_as_shapely_polygons' attribute of the database.

    Note: If multiple ROIs are used for one image, the individual ROIs must be named properly in the ROIManager-Tool in ImageJ.
          For instance, if images of the hippocampus are investigated & they can contain images of the DG, CA3 and CA1, 
          the corresponding ROIs that mark the respective area have to be named consistenly for all .zip files. This makes it possible, 
          that findmycells can handle the analysis even if not all ROIs are present for each image, e.g. for some files only DG and CA3.
    """
    
    
    @property
    def key_to_configs_attribute_in_database(self) -> str:
        return 'ROIReader_configs'
    
    
    @property
    def default_config_values(self) -> Dict[str, Any]:
        """
        findmycells enables analyses of multiple ROIs in the image data. To do so, they will be matched based on their ID that will
        be retrieved from the ROI file. Some softwares that create these ROI-files, however, create default IDs for the individual
        ROIs that will interfere with this matching. For instance, in Fiji / ImageJ2, created ROIs get its centroid (?) pixel 
        coordinates as default ID (e.g. something like "523-378"). Since such default IDs most likely won´t be consistent throughout
        your entire image dataset, findmycells provides you with two options to adress this:
        a) You can set 'load_roi_ids_from_file' to False (default): 
          This will cause findmycells to ignore the IDs of the ROIs that are saved in the provided ROI file and assign them with 
          new IDs starting at "000". Note: Essentially, this requires you to have always the same type of ROIs present in the 
          exact same order in all your ROI-files. It is therefore only recommended if you have just a single ROI you´d like to analyze.
        b) You can set 'load_roi_ids_from_file' to True (recommended if you have more than a single ROI):
          This will enforce that findmycells uses the IDs that each ROI was saved with. Therefore, it requires that you use consistent
          naming of the ROIs with your preferred software. For instance, if you´re using Fiji / ImageJ2, you can rename each ROI in the
          ROIManager (e.g. "CA3", "vlPAG", or "ipsilateral_SNc"). Analyses and quantifications will then be matched and pooled across
          all ROIs with the respective IDs (e.g. all "CA3" ROIs).
        """
        default_values = {'load_roi_ids_from_file': False}
        return default_values
    
    
    @property
    def default_config_value_types(self) -> Dict[str, List[type]]:
        default_config_value_types = {'load_roi_ids_from_file': [bool]}
        return default_config_value_types
    
    
    def assert_correct_output_format(self, output: Dict[str, Dict[str, Polygon]]) -> None:
        assert type(output) == dict, 'The overall type of the returned data is not a dictionary!'
        for plane_id, nested_dict in output.items():
            assert type(plane_id) == str, 'Not all keys of the constructed ROI dictrionary are strings!'
            assert type(nested_dict) == dict, 'Not all elements in the constructed ROI dictionary are nested dictionaries!'
            for roi_id, polygon in output[plane_id].items():
                assert type(roi_id) == str, 'Not all assigned ROI-IDs are strings!'
                assert type(polygon) == Polygon, 'Not all loaded ROIs were successfully converted into Polygon objects!'

In [ ]:
#| export

class ImageJROIReader(ROIReaders):
    
    
    @property
    def readable_filetype_extensions(self) -> List[str]:
        return ['.roi', '.zip']
    
    
    def read(self,
             filepath: Path, # filepath to the microscopy image file
             database: Database # the project database
            ) -> Dict[str, Dict[str, Polygon]]: # nested dictionaries of shapely polygons: {plane_id: {roi_id: Polygon}}
        if filepath.name.endswith('.roi'):
            loaded_rois = [roifile.ImagejRoi.fromfile(filepath)]
        else: # it´s a .zip file:
            loaded_rois = roifile.ImagejRoi.fromfile(filepath)
        rois_as_shapely_polygons = {'all_planes': {}} # plane specific ROIs are not yet supported, but this structure would allow it
        roi_count = len(loaded_rois)
        for idx in range(roi_count):
            row_coords = loaded_rois[idx].coordinates()[:, 1]
            col_coords = loaded_rois[idx].coordinates()[:, 0]
            if self.load_roi_ids_from_file:
                rois_as_shapely_polygons['all_planes'][loaded_rois[idx].name] = Polygon(np.asarray(list(zip(row_coords, col_coords))))
            else:
                rois_as_shapely_polygons['all_planes'][str(idx).zfill(3)] = Polygon(np.asarray(list(zip(row_coords, col_coords))))            
        return rois_as_shapely_polygons

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()